In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [181]:
import itertools
from operator import itemgetter
import numpy as np
from scipy.optimize import linprog
from cvxopt import matrix, solvers, spmatrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from initialize import linparams

In [3]:
# Parameter values
n_class = 2
los = 3
capacity = 50
intensity = 1.5
slope_init = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])
combs = n_class * 7 * los

In [8]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [9]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [10]:
# Coefficients of objective function for LP
obj_coefs = (-1) * rates_arrival_los.reshape(n_class * 7 * los)
obj_coefs

array([-135, -270, -405, -135, -270, -405, -135, -270, -405, -135, -270,
       -378, -135, -243, -351, -108, -216, -351, -108, -243, -378, -115,
       -230, -345, -115, -230, -345, -115, -230, -345, -115, -230, -322,
       -115, -207, -299,  -92, -184, -299,  -92, -207, -322])

In [11]:
# Inequality equations, LHS
# We have total number of 42 decision veriables, corresponding to total number of
# rate class, arrival day of week and los combinations.
# Column indexes 0-20 are associated with decision variables for rate class 1
# Column indexes 21-41 are associated with decision variables for rate class 2
G = np.zeros(7 * los * n_class * 7).reshape(7, n_class*7*los)
# Arrivals that span Sunday stay night for rate class 1
G[0,:(7*los)] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 1
G[1,:(7*los)] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,:(7*los)] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,:(7*los)] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
# Arrivals that span Sunday stay night for rate class 2
G[0,(7*los):] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 2
G[1,(7*los):] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,(7*los):] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,(7*los):] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]

# identity matrix for expected demand constraints, G for capacity constraints,
# Negative identity matrix for non-negativity
G = np.concatenate((np.identity(combs), G, -np.identity(combs)), axis=0)
# Inequality equations, RHS
# For each rate class, number of arrivals for a stay night in question is half of
# expected demand, which is capacity * intensity, then this expected demand is equally 
# split between 6 arrival day, los combination that spans the stay night in question
expDemand_each = (capacity * intensity * 0.5) / 6
h = np.round(expDemand_each, decimals=0) * np.ones(n_class * 7 * los)
# First h for expected demand, second component for capacity rhs
# Third component for non-negativity rhs.
h = np.concatenate((h, capacity * np.ones(7), np.zeros(combs)), axis=0)

In [12]:
h

array([ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6., 50., 50., 50., 50., 50., 50., 50.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [13]:
# Convert numpy arrays to cvxopt matrix forms
c = matrix(obj_coefs, tc='d')
G = matrix(G)
h = matrix(h)

In [186]:
# Solve LP, method = "interior point method" by default
# Results are used for initialization purpose (warm start), so it should
# not affect final algorithm performance after an efficient number of runs
sol = solvers.lp(c, G, h)

     pcost       dcost       gap    pres   dres   k/t
 0: -4.3576e+04 -1.0770e+05  5e+04  1e-01  8e-01  1e+00
 1: -4.4750e+04 -7.3030e+04  2e+04  7e-02  4e-01  5e+01
 2: -4.5974e+04 -5.1996e+04  3e+03  1e-02  8e-02  2e+01
 3: -4.6445e+04 -4.7593e+04  6e+02  3e-03  1e-02  3e+00
 4: -4.6520e+04 -4.6801e+04  2e+02  7e-04  4e-03  1e+00
 5: -4.6547e+04 -4.6554e+04  3e+00  1e-05  8e-05  3e-02
 6: -4.6548e+04 -4.6548e+04  3e-02  1e-07  8e-07  3e-04
 7: -4.6548e+04 -4.6548e+04  3e-04  1e-09  8e-09  3e-06
Optimal solution found.


In [192]:
# Optimal solutions serve as booking limits: number of booking requests to accept for 
# a given rate class, arrival day, los combination
bkLimits = np.array(sol['x']).reshape(n_class, 7, los)
bkLimits = np.round(bkLimits, decimals=0)
bkLimits

array([[[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [2., 6., 6.]],

       [[6., 6., 6.],
        [4., 2., 2.],
        [1., 1., 2.],
        [3., 3., 3.],
        [3., 3., 0.],
        [2., 0., 6.],
        [0., 6., 6.]]])

In [16]:
# Dual values associated with demand constraints
# Represent marginal contribution for the stay night revenue
duals = np.array(sol['z'])[:(n_class*7*los)].reshape(n_class, 7, los)
duals = np.round(duals, decimals=0)
duals

array([[[135., 155., 175.],
        [ 20.,  40.,  60.],
        [ 20.,  40.,  60.],
        [ 20.,  40.,  56.],
        [ 20.,  36.,  36.],
        [ 16.,  16., 151.],
        [  0., 135., 270.]],

       [[115., 115., 115.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,  99.],
        [  0.,  99., 214.]]])

In [17]:
# tuple (0, 1, 2) represents rate class 1, Monday arrival and 3-night stay
sun_stay_index = [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 6, 1), (0, 6, 2), (0, 5, 2), 
            (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 6, 1), (1, 6, 2), (1, 5, 2)]

mon_stay_index = [(0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 0, 1), (0, 0, 2), (0, 6, 2), 
            (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 0, 1), (1, 0, 2), (1, 6, 2)]

tue_stay_index = [(0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 1, 1), (0, 1, 2), (0, 0, 2), 
            (1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 1, 1), (1, 1, 2), (1, 0, 2)]

wed_stay_index = [(0, 3, 0), (0, 3, 1), (0, 3, 2), (0, 2, 1), (0, 2, 2), (0, 1, 2), 
            (1, 3, 0), (1, 3, 1), (1, 3, 2), (1, 2, 1), (1, 2, 2), (1, 1, 2)]

thr_stay_index = [(0, 4, 0), (0, 4, 1), (0, 4, 2), (0, 3, 1), (0, 3, 2), (0, 2, 2), 
            (1, 4, 0), (1, 4, 1), (1, 4, 2), (1, 3, 1), (1, 3, 2), (1, 2, 2)]

fri_stay_index = [(0, 5, 0), (0, 5, 1), (0, 5, 2), (0, 4, 1), (0, 4, 2), (0, 3, 2), 
            (1, 5, 0), (1, 5, 1), (1, 5, 2), (1, 4, 1), (1, 4, 2), (1, 3, 2)]

sat_stay_index = [(0, 6, 0), (0, 6, 1), (0, 6, 2), (0, 5, 1), (0, 5, 2), (0, 4, 2), 
            (1, 6, 0), (1, 6, 1), (1, 6, 2), (1, 5, 1), (1, 5, 2), (1, 4, 2)]

In [18]:
# Create one complete index list
stay_index = [sun_stay_index, mon_stay_index, tue_stay_index, wed_stay_index,
             thr_stay_index, fri_stay_index, sat_stay_index]

In [23]:
# Create virtual buckets with booking classes in them
# Use a copy of the index list in order to keep the index list unchanged for later use
stay_index_cp = stay_index.copy()
max_buckets = 5
buckets_all = []

for index_ls in stay_index_cp:
    buckets = [[] for i in range(max_buckets)]
    duals_ls = [duals[item] for item in index_ls]
    n_buckets = 1

    # Stop clustering when the number of buckets is higher than max_buckets of 5 or
    # when the index list is empty
    while n_buckets <= max_buckets and index_ls:
        duals_left = [duals[item] for item in index_ls]
        duals_max = np.max(duals_left)
        # Keep track of the elements to remove from the index list in 
        # each bucket creation iteration
        item_delete_index = []
        # Create lower and upper limits for buckets
        lower = ((max_buckets-n_buckets) / (max_buckets-n_buckets+1)) * duals_max
        upper = duals_max
        # Cluster each item into appropriate buckets
        for item in index_ls:
            item_index = index_ls.index(item)
            if duals[item] >= lower: 
                buckets[n_buckets-1].append(item)
                item_delete_index.append(item_index)
        # Update index list for the next iteration by removing elements that have been
        # already clustered
        index_ls = [item for item in index_ls if index_ls.index(item) not in item_delete_index]
        # Update bucket number
        n_buckets += 1
        
    buckets_all.append(buckets)
    

In [24]:
# Filter out empty lists where number of buckets for a stay night is less than 5
buckets_all = [list(filter(None, elem)) for elem in buckets_all]

In [129]:
# Cluster booking classes that span Sunday stay night into 4 buckets
buckets_all[0]

[[(0, 6, 2)],
 [(0, 0, 2), (1, 6, 2)],
 [(0, 0, 0), (0, 0, 1), (0, 6, 1), (0, 5, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2)],
 [(1, 6, 1), (1, 5, 2)]]

In [26]:
# Number of buckets in each stay night
n_buckets = list(map(len, buckets_all))
n_buckets

[4, 5, 5, 4, 4, 5, 5]

In [210]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [211]:
bkLimits

array([[[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [2., 6., 6.]],

       [[6., 6., 6.],
        [4., 2., 2.],
        [1., 1., 2.],
        [3., 3., 3.],
        [3., 3., 0.],
        [2., 0., 6.],
        [0., 6., 6.]]])

In [214]:
buckets_all[0][0][0]

(0, 6, 2)

In [217]:
# Calculate protection levels for each bucket in each stay night of the week
# Store partitioned protection levels for all days of a week
ptLevels_ptd = []
for buckets_each in buckets_all:
    # Store partitioned protection levels for each stay night
    levels = []
    # Partitioned protection levels
    for bucket in buckets_each:
        # If there is only one element in the bucket, then it is not iterable
        # and code will throw out TypeError. Therefore, we implement try...except.
        try:
            level = list(itemgetter(*bucket)(bkLimits))
            level = np.sum(np.array(level).reshape(len(level)))
        except TypeError:
            level = bkLimits[bucket[0]]
        levels.append(level)
    ptLevels_ptd.append(levels)

In [218]:
ptLevels_ptd

[[6.0, 12.0, 42.0, 12.0],
 [6.0, 12.0, 18.0, 12.0, 14.0],
 [6.0, 6.0, 24.0, 6.0, 8.0],
 [18.0, 12.0, 6.0, 14.0],
 [12.0, 18.0, 6.0, 14.0],
 [6.0, 6.0, 6.0, 12.0, 20.0],
 [6.0, 6.0, 12.0, 12.0, 14.0]]

In [219]:
# Compute nested protection levels
ptLevels = [np.cumsum(ptLevels_ptd[i]) for i in range(7)]
ptLevels

[array([ 6., 18., 60., 72.]),
 array([ 6., 18., 36., 48., 62.]),
 array([ 6., 12., 36., 42., 50.]),
 array([18., 30., 36., 50.]),
 array([12., 30., 36., 50.]),
 array([ 6., 12., 18., 30., 50.]),
 array([ 6., 12., 24., 36., 50.])]

In [233]:
# Store number of rda combinations in a bucket for a stay night
bkt_length = [list(map(len, buckets_all[i])) for i in range(7)]
bkt_length

[[1, 2, 7, 2],
 [1, 2, 3, 2, 4],
 [1, 1, 4, 1, 5],
 [3, 2, 1, 6],
 [2, 3, 1, 6],
 [1, 1, 1, 2, 7],
 [1, 1, 2, 2, 6]]

In [222]:
# Calculate average revenue for each bucket in each stay night of the week
# Store calculated average revenues for all days of a week
revenues_avg = []
for buckets_each in buckets_all:
    # Store average revenue for each stay night
    revenue_each = []
    for bucket in buckets_each:
        # If there is only one element in the bucket, then it is not iterable
        # and code will throw out TypeError. Therefore, we implement try...except.
        try:
            revenue = list(itemgetter(*bucket)(rates_arrival_los))
            revenue = np.mean(np.array(revenue).reshape(len(revenue)))
        except TypeError:
            revenue = rates_arrival_los[bucket[0]]
        revenue_each.append(revenue)
    revenues_avg.append(revenue_each)

In [224]:
revenues_avg

[[378, 363.5, 241.28571428571428, 253.0],
 [378, 363.5, 281.6666666666667, 337.5, 206.25],
 [405, 345, 337.5, 135, 253.0],
 [396.0, 270.0, 135, 264.5],
 [391.5, 288.0, 135, 253.0],
 [351, 299, 378, 297.0, 204.0],
 [378, 322, 297.0, 253.0, 208.33333333333334]]

In [230]:
# Calculate minimum dual values for each bucket in each stay night of the week
# Store output for all days of a week
duals_min = []
for buckets_each in buckets_all:
    # Store average revenue for each stay night
    dual_vals = []
    for bucket in buckets_each:
        # If there is only one element in the bucket, then it is not iterable
        # and code will throw out TypeError. Therefore, we implement try...except.
        try:
            dual_val = list(itemgetter(*bucket)(duals))
            dual_val = np.min(np.array(dual_val).reshape(len(dual_val)))
        except TypeError:
            dual_val = duals[bucket[0]]
        dual_vals.append(dual_val)
    duals_min.append(dual_vals)

In [237]:
duals_min, revenues_avg

([[270.0, 175.0, 115.0, 99.0],
  [270.0, 175.0, 115.0, 40.0, 0.0],
  [175.0, 115.0, 40.0, 20.0, 0.0],
  [56.0, 40.0, 20.0, 0.0],
  [56.0, 36.0, 20.0, 0.0],
  [151.0, 99.0, 56.0, 36.0, 0.0],
  [270.0, 214.0, 135.0, 99.0, 0.0]],
 [[378, 363.5, 241.28571428571428, 253.0],
  [378, 363.5, 281.6666666666667, 337.5, 206.25],
  [405, 345, 337.5, 135, 253.0],
  [396.0, 270.0, 135, 264.5],
  [391.5, 288.0, 135, 253.0],
  [351, 299, 378, 297.0, 204.0],
  [378, 322, 297.0, 253.0, 208.33333333333334]])

In [250]:
# Calculate dual values difference and find the average difference
# Here, implement Algorithm 2 from the paper
duals_diff = []
for vals in duals_min:
    duals_diff_each = []
    for i in range(len(vals) - 1):
        dual_diff = vals[i] - vals[i+1]
        duals_diff_each.append(dual_diff)
    duals_diff.append(duals_diff_each)

In [251]:
duals_diff

[[95.0, 60.0, 16.0],
 [95.0, 60.0, 75.0, 40.0],
 [60.0, 75.0, 20.0, 20.0],
 [16.0, 20.0, 20.0],
 [20.0, 16.0, 20.0],
 [52.0, 43.0, 20.0, 36.0],
 [56.0, 79.0, 36.0, 99.0]]